In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
# import tensorflow
# from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

In [5]:
train = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/train_asos(예보 제외).csv', encoding = 'cp949')
test = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/test_asos(예보 제외).csv', encoding = 'cp949')


In [6]:
#date time으로 변환
train['yyyymmdd'] = pd.to_datetime(train['yyyymmdd'])
test['yyyymmdd'] = pd.to_datetime(test['yyyymmdd'])

# MinMax 정규화

In [8]:
scaler = MinMaxScaler()
train.iloc[:,4:] = scaler.fit_transform(train.iloc[:,4:])

print(train['평균_기온'].min())
print(train['평균_기온'].max())

0.0
1.0


In [9]:
scaler = MinMaxScaler()
test.iloc[:,4:] = scaler.fit_transform(test.iloc[:,4:])

print(test['평균_기온'].min())
print(test['평균_기온'].max())

0.0
1.0


# 원핫 인코딩(성별)

In [10]:
#train 성별
train['성별(남)'] = 0
train['성별(여)'] = 0

train.loc[train['sex'] == 1, '성별(남)'] = 1
train.loc[train['sex'] == 2, '성별(남)'] = 0
train.loc[train['sex'] == 1, '성별(여)'] = 0
train.loc[train['sex'] == 2, '성별(여)'] = 1

In [11]:
#test 성별
test['성별(남)'] = 0
test['성별(여)'] = 0

test.loc[test['sex'] == 1, '성별(남)'] = 1
test.loc[test['sex'] == 2, '성별(남)'] = 0
test.loc[test['sex'] == 1, '성별(여)'] = 0
test.loc[test['sex'] == 2, '성별(여)'] = 1

#원핫 인코딩(지역)

In [12]:
# train 지역 칼럼 생성
for i in train['area'].unique():
    train[i] = 0

In [13]:
# train 지역 원핫 인코딩
for i in train['area'].unique():
    train.loc[train['area'] == i, i] = 1

In [14]:
# test 지역 칼럼 생성
for i in test['area'].unique():
    test[i] = 0

In [15]:
# test 지역 원핫 인코딩
for i in test['area'].unique():
    test.loc[test['area'] == i, i] = 1

#fb-Prophet

In [16]:
# Prophet Library 
from fbprophet import Prophet 
# Prophet 시각화 도구 
from fbprophet.plot import plot_plotly, plot_components_plotly
from fbprophet.plot import add_changepoints_to_plot


In [17]:
train_fb = train.drop(columns=['area','sex'])
train_fb

,yyyymmdd,frequency,평균_기온,최저_기온,최고_기온,일교차,평균_지면온도,평균_강수량,평균_풍속,평균_상대습도,...,부산,서울,울산,인천,전남,전북,제주,충남,충북,세종
0,2012-01-01,3,0.250420,0.288098,0.228778,0.289463,0.109720,0.000606,0.146383,0.578217,...,0,0,0,0,0,0,0,0,0,0
1,2012-01-01,3,0.250420,0.288098,0.228778,0.289463,0.109720,0.000606,0.146383,0.578217,...,0,0,0,0,0,0,0,0,0,0
2,2012-01-01,4,0.226471,0.256534,0.211594,0.319803,0.103880,0.000000,0.112340,0.676298,...,0,0,0,0,0,0,0,0,0,0
3,2012-01-01,5,0.226471,0.256534,0.211594,0.319803,0.103880,0.000000,0.112340,0.676298,...,0,0,0,0,0,0,0,0,0,0
4,2012-01-01,2,0.328992,0.355649,0.334369,0.360804,0.153942,0.000173,0.157447,0.540971,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49305,2015-12-31,0,0.479412,0.543225,0.419255,0.146781,0.301210,0.001515,0.374468,0.579571,...,0,0,0,0,0,0,1,0,0,0
49306,2015-12-31,0,0.328992,0.363088,0.319669,0.316523,0.196496,0.000173,0.074894,0.845147,...,0,0,0,0,0,0,0,1,0,0
49307,2015-12-31,0,0.328992,0.363088,0.319669,0.316523,0.196496,0.000173,0.074894,0.845147,...,0,0,0,0,0,0,0,1,0,0
49308,2015-12-31,1,0.314706,0.347004,0.280331,0.271423,0.168127,0.000693,0.091915,0.764108,...,0,0,0,0,0,0,0,0,1,0


In [18]:
y_train_prophet_df = train_fb[['yyyymmdd','frequency']]
y_train_prophet_df.columns = ['ds', 'y']
y_train_prophet_df.head()

,ds,y
0,2012-01-01,3
1,2012-01-01,3
2,2012-01-01,4
3,2012-01-01,5
4,2012-01-01,2


In [19]:
x_col = [i for i in train_fb.columns if i not in ['yyyymmdd','frequency']]
x_train_prophet_df = train_fb[x_col]
x_train_prophet_df.head()


,평균_기온,최저_기온,최고_기온,일교차,평균_지면온도,평균_강수량,평균_풍속,평균_상대습도,평균_현지기압,평균_해면기압,...,부산,서울,울산,인천,전남,전북,제주,충남,충북,세종
0,0.250420,0.288098,0.228778,0.289463,0.109720,0.000606,0.146383,0.578217,0.416108,0.690540,...,0,0,0,0,0,0,0,0,0,0
1,0.250420,0.288098,0.228778,0.289463,0.109720,0.000606,0.146383,0.578217,0.416108,0.690540,...,0,0,0,0,0,0,0,0,0,0
2,0.226471,0.256534,0.211594,0.319803,0.103880,0.000000,0.112340,0.676298,0.747804,0.729042,...,0,0,0,0,0,0,0,0,0,0
3,0.226471,0.256534,0.211594,0.319803,0.103880,0.000000,0.112340,0.676298,0.747804,0.729042,...,0,0,0,0,0,0,0,0,0,0
4,0.328992,0.355649,0.334369,0.360804,0.153942,0.000173,0.157447,0.540971,0.713265,0.686193,...,0,0,0,0,0,0,0,0,0,0


In [20]:
yx_train_prophet_df = pd.concat(
    [y_train_prophet_df, x_train_prophet_df],
    axis=1
    )

In [21]:
# Applying Prophet Model 
fit_default_prophet = Prophet(
    # 1) linear, nonlinear
    growth='linear',
    
    # 2) Trend 
    changepoints=None,# CP가 발생하는 시점들의 list ['2012-01-01'] 
    
    n_changepoints=25, # CP의 수
    changepoint_range=0.8, # CP의 기존 데이터 수 대비 최대 비율
    changepoint_prior_scale=0.05, # CP 추정 민감도로 높을수록 민감
    
    # 3) Seasonality 
    seasonality_mode='additive', # 계절성 모델: 'additive' or 'multiplicative'
    seasonality_prior_scale=10.0, # 계절성 추정 민감도로 높을수록 민감
    yearly_seasonality='auto', # 연계절성
    weekly_seasonality='auto',# 월계절성
    daily_seasonality='auto', #일계절성
    
    # 4) Holiday
    holidays=None, # 휴일 또는 이벤트 시점
    holidays_prior_scale=10.0, # 휴일 추정 민감도로 높을수록 민감
    
    # 5) Others
    interval_width=0.8, # 추세 예측 정확도 구간범위
    mcmc_samples=0, # 계절성 예측 정확도 제어
    )

In [24]:
# model 학습 
fit_reg1_prophet = fit_default_prophet.fit(yx_train_prophet_df) 

# Frequency에 대한 사전 df를 만듦 
forecast = fit_reg1_prophet.make_future_dataframe(
    freq='H',
    periods=sub.shape[0]
    ) 

# predict pred_reg1_prophet = fit_reg1_prophet.predict(forecast) 

pred_train_reg1_prophet = np.ravel(pred_reg1_prophet.loc[:yx_train_prophet_df.shape[0]-1, ['yhat']])
pred_test_reg1_prophet = np.ravel(pred_reg1_prophet.loc[yx_train_prophet_df.shape[0]:, ['yhat']])


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


NameError: ignored

In [ ]:
sub['count'] = pred_test_reg1_prophet sub['count'] = sub['count'].apply(lambda x:round(x,0)) sub.loc[sub['count']<0,'count'] = 0 sub.head()
출처: https://today-1.tistory.com/41 [Better Than,:티스토리]